In [1]:
import pickle
from collections import Counter
import keras
#import postprocessing as pr
import numpy as np
import logging
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# with open('/content/drive/My Drive/dataset/dataset/generation_dataset.pickle', 'rb') as pickle_file:
#     generation_dataset = pickle.load(pickle_file)
#     articles = generation_dataset["article_text"]
#     headlines = generation_dataset["title"]
    # ###GET COUNT
    # from collections import Counter
    # from nltk import FreqDist
    
    # word_dist = FreqDist()
    # for s in articles:
    #     word_dist.update(s.split())
    # for s in headlines:
    #   word_dist.update(s.split())
    # counter =  Counter(word_dist)
    # with open("/content/drive/My Drive/dataset/dataset/vocab.dict.50000", "w") as f:
    #     f.write("".join([ w+"\t"+str(c)+"\n"  for w, c in counter.most_common(50000)]))

In [2]:
with open('/content/drive/My Drive/dataset/dataset/generation_dataset.pickle', 'rb') as pickle_file:
    generation_dataset = pickle.load(pickle_file)
    articles = generation_dataset["article_text"]
    headlines = generation_dataset["title"]

FileNotFoundError: ignored

Vocab

In [0]:
# def getvocab(lst):
#   vocabCount, vocab = Counter(w for txt in lst for w in txt.split())
#   return vocab, vocabCount
# vocab, vocabCount = getvocab(articles.head()+headlines.head())
# print(vocab[:50])
# print('...', len(vocab))

# ###GET COUNT
# from collections import Counter
# from nltk import FreqDist

# word_dist = FreqDist()
# for s in articles.head():
#     word_dist.update(s.split())
# for s in headlines.head():
#   word_dist.update(s.split())
# counter =  Counter(word_dist)
# # with open("/content/drive/My Drive/dataset/dataset/vocab.dict.50000", "w") as f:
# #     f.write("".join([ w+"\t"+str(c)+"\n"  for w, c in counter.most_common(50000)]))
# print(word_dist)

<FreqDist with 2004 samples and 5642 outcomes>


In [0]:
FN = 'vocabulary-embedding'
seed=42
vocab_size = 50000
embedding_dim = 300

In [0]:
from collections import Counter
from itertools import chain
def get_vocab(lst):
    vocabcount = Counter(w for txt in lst for w in txt.split())
    vocab = map(lambda x: x[0], sorted(vocabcount.items(), key=lambda x: -x[1]))
    return vocab, vocabcount

In [0]:
vocab, vocabcount = get_vocab(articles+headlines)
#vocab2, vocabcount2 = get_vocab(articles.head()+headlines.head())
#print(list(vocab2))
# print(vocabcount)

Index words

In [0]:
empty = 0 # RNN mask of no data
eos = 1  # end of sentence
start_idx = eos+1 # first real word

In [0]:
def get_idx(vocab, vocabcount):
    word2idx = dict((word, idx+start_idx) for idx,word in enumerate(vocab))
    word2idx['<empty>'] = empty
    word2idx['<eos>'] = eos
    
    idx2word = dict((idx,word) for word,idx in word2idx.items())

    return word2idx, idx2word

In [0]:
word2idx, idx2word = get_idx(vocab, vocabcount)
#print(word2idx['$numberplaceholder'])
#word2idx['numberplaceholder']
#print(idx2word)

Word Embedding

read Glove

In [0]:
!wget https://docs.google.com/uc?id=1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu -O glove_split.aa
!wget https://docs.google.com/uc?id=1LF2yD2jToXriyD-lsYA5hj03f7J3ZKaY -O glove_split.ab
!wget https://docs.google.com/uc?id=1N1xnxkRyM5Gar7sv4d41alyTL92Iip3f -O glove_split.ac
!cat glove_split.?? > 'glove.6B.300d__50k.txt'

--2020-05-13 17:15:50--  https://docs.google.com/uc?id=1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu
Resolving docs.google.com (docs.google.com)... 173.194.216.139, 173.194.216.100, 173.194.216.138, ...
Connecting to docs.google.com (docs.google.com)|173.194.216.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5sb87i4do2735304iaieli698f21qll0/1589390100000/14514704803973256873/*/1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu [following]
--2020-05-13 17:15:55--  https://doc-0k-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5sb87i4do2735304iaieli698f21qll0/1589390100000/14514704803973256873/*/1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu
Resolving doc-0k-0g-docs.googleusercontent.com (doc-0k-0g-docs.googleusercontent.com)... 173.194.216.132, 2607:f8b0:400c:c12::84
Connecting to doc-0k-0g-docs.googleusercontent.com (doc-0k-0g-docs.googleusercontent.com)|173.19

In [0]:
def load_glove(glove_path, embedding_dim):
    with open(glove_path) as f:
        token_ls = [PAD_TOKEN, UNK_TOKEN]
        embedding_ls = [np.zeros(embedding_dim), np.random.rand(embedding_dim)]
        for line in f:
            token, raw_embedding = line.split(maxsplit=1)
            token_ls.append(token)
            embedding = np.array([float(x) for x in raw_embedding.split()])
            embedding_ls.append(embedding)
        embeddings = np.array(embedding_ls)
    return token_ls, embeddings

PAD_TOKEN = '<PAD>'
UNK_TOKEN = '<UNK>'
glove_path = "glove.6B.300d__50k.txt"

vocab, embeddings = load_glove(glove_path, embedding_dim)

In [0]:
glove_n_symbols = !wc -l {glove_path}
glove_n_symbols = int(glove_n_symbols[0].split()[0])
glove_n_symbols

50000

In [0]:
glove_index_dict = {}
glove_embedding_weights = np.empty((glove_n_symbols, embedding_dim))
globale_scale=.1
with open(glove_path, 'r') as fp:
    i = 0
    for l in fp:
        l = l.strip().split()
        w = l[0]
        glove_index_dict[w] = i
        glove_embedding_weights[i,:] = list(map(float,l[1:]))
        i += 1
glove_embedding_weights *= globale_scale

In [0]:
glove_embedding_weights.std()

0.038998941967345574

In [0]:
for w,i in glove_index_dict.items():
    w = w.lower()
    if w not in glove_index_dict:
        glove_index_dict[w] = i

embedding matrix

In [0]:
# generate random embedding with same scale as glove
np.random.seed(seed)
shape = (vocab_size, embedding_dim)
scale = glove_embedding_weights.std()*np.sqrt(12)/2 # uniform and not normal
embedding = np.random.uniform(low=-scale, high=scale, size=shape)
print('random-embedding/glove scale', scale, 'std', embedding.std())

# copy from glove weights of words that appear in our short vocabulary (idx2word)
c = 0
for i in range(vocab_size):
    w = idx2word[i]
    g = glove_index_dict.get(w, glove_index_dict.get(w.lower()))
    if g is None and w.startswith('#'): # glove has no hastags (I think...)
        w = w[1:]
        g = glove_index_dict.get(w, glove_index_dict.get(w.lower()))
    if g is not None:
        embedding[i,:] = glove_embedding_weights[g,:]
        c+=1
print("number of tokens, in small vocab, found in glove and copied to embedding", c, c/float(vocab_size))

random-embedding/glove scale 0.06754814892887268 std 0.03899395375485842
number of tokens, in small vocab, found in glove and copied to embedding 26554 0.53108


In [0]:
glove_thr = 0.4

In [0]:
word2glove = {}
for w in word2idx:
    if w in glove_index_dict:
        g = w
    elif w.lower() in glove_index_dict:
        g = w.lower()
    elif w.startswith('#') and w[1:] in glove_index_dict:
        g = w[1:]
    elif w.startswith('#') and w[1:].lower() in glove_index_dict:
        g = w[1:].lower()
    else:
        continue
    word2glove[w] = g

In [0]:
normed_embedding = embedding/np.array([np.sqrt(np.dot(gweight,gweight)) for gweight in embedding])[:,None]

nb_unknown_words = 100

glove_match = []
for w,idx in word2idx.items():
    if idx >= vocab_size-nb_unknown_words and w.isalpha() and w in word2glove:
        gidx = glove_index_dict[word2glove[w]]
        gweight = glove_embedding_weights[gidx,:].copy()
        # find row in embedding that has the highest cos score with gweight
        gweight /= np.sqrt(np.dot(gweight,gweight))
        score = np.dot(normed_embedding[:vocab_size-nb_unknown_words], gweight)
        while True:
            embedding_idx = score.argmax()
            s = score[embedding_idx]
            if s < glove_thr:
                break
            if idx2word[embedding_idx] in word2glove :
                glove_match.append((w, embedding_idx, s)) 
                break
            score[embedding_idx] = -1
glove_match.sort(key = lambda x: -x[2])
print('# of glove substitutes found', len(glove_match))

# of glove substitutes found 15737


In [0]:
for orig, sub, score in glove_match[-10:]:
    print(score, orig,'=>', idx2word[sub])

0.4002781050070189 zivkovic => viktor
0.4002129276288864 loc => kashmir
0.40016366302851203 bimbo => grupo
0.4001619968486482 padre => tierra
0.4001500725941828 whitacre => daniels
0.40014388986529986 riad => saeed
0.4000905262097927 jonsson => lars
0.4000658687107316 ornstein => scholar
0.4000390560983827 banach => algebra
0.4000218390742082 foy => vance


In [0]:
glove_idx2idx = dict((word2idx[w],embedding_idx) for  w, embedding_idx, _ in glove_match)

Data

In [0]:
Y = [[word2idx.get(token, None) for token in d.split()] for d in headlines]
len(Y)

375000

In [0]:
X = [[word2idx.get(token, None) for token in d.split()] for d in articles]
len(X)

375000

In [0]:
!ls '/content/drive/My Drive/dataset/headline_generator/outputs'

In [0]:
with open('/content/drive/My Drive/dataset/headline_generator/outputs/%s.pkl'%FN,'wb') as fp:
    pickle.dump((embedding, idx2word, word2idx, glove_idx2idx),fp,-1)

In [0]:
with open('/content/drive/My Drive/dataset/headline_generator/outputs/%s.data.pkl'%FN,'wb') as fp:
    pickle.dump((X,Y),fp,-1)